In [ ]:
#Need to run get_trench_perp_profiles_regional first to generate the segments portion of the file to be read in at the end
# This file is setup to read test_slab2wbspherical_output.wb which is a copy of vep_noadbtnoWC.wb 

import json
import numpy as np
import pygmt
import math

# Read in a WorldBuilder File
#Uses most recent version of rfildes worldbuilder file to serve as template to be filled in
worldbuilder_json_file = 'vep_slab8060WC.wb'

file_object = open(worldbuilder_json_file,'r')
wbdict = json.load(file_object)
print('The template worldbuilder file has been read in as a ', type(wbdict))

#### Layout of the plates

Overriding1

Overriding2: between the spreading center and the slab

Ridge-Corner-Upper & Ridge-Corner-Sub: weak region in the corner

SlabTemp: temperature of the slab, separeted to create a tappering at the end.

SlabComp: composition of the slab

![](./05132024_0.png)

In [ ]:
# Order of Features in this World Builder File (for Kermadec plate geometry) is:
    # [0] Plate C (East) (Overrriding Plate 1)); models: temperature, composition
    # [1] Plate B (Middle) (Overrriding Plate 2)); models: temperature, composition
    # [2] WC OP:models: temp, comp
    # [3] Plate A (West)(Subducting); models: temperature, composition
    # [4] WC SP; models: temp, comp
    # [5] SlabTemp; models: temperature
    # [6] SlabComp; models: composition
    
    #Note, because B and C share a ridge, the variable names often lump together "BC", if the variable is the same for both
# Compositional layers: crust and harzburgite -> model: uniform
# Plate temperature model: half space cooling
# Slab temperature model: mass conserving

# Directions are dist (degrees), depth (z), out-of-plane (y)
# meaning of dy1 dy2, dy3:
# the redundant coordinate in 2d world builder
dy1 = -5  # width in out-of plane section
dy2 =  5
dy3 = -10
trench_loc = 44 #degrees
ridgeBC_loc = trench_loc-2.00 #degrees
# Location (on map) information that is need for 2D cross section
# Cross Section Coordinates [[start_dist, start_depth], [end_dist, end_depth]]
start_dist = 0
start_depth = 0
end_dist = 80 #degrees 
end_depth = 0

#########%%%%%%%%%REST OF THIS CELL NEEDS TO BE DOUBLE CHECKED for subduction direction%%%%%%%
dip_direction = 0; #  
if dip_direction == 0:
    dip_point = [start_dist, 0]  # dips north or west, Plate A is subducting
else:
    dip_point = [end_dist, 0]  # dips south or east, Plate B is subducting

In [ ]:
# Orientation of 2D cross section is determined by end points of a profiles

wbdict['cross section'] = [[start_dist,start_depth],[end_dist,end_depth]]

# Points outlining map geometry of a plate: in 2D these are rectangles
#plateC - OP1
wbdict['features'][0]['coordinates'] = \
    [ [start_dist, dy1], [start_dist, dy2], [ridgeBC_loc, dy2], [ridgeBC_loc, dy1] ]

#plateB  -OP2 
wbdict['features'][1]['coordinates'] = \
    [ [ridgeBC_loc, dy1], [ridgeBC_loc, dy2], [trench_loc, dy2], [trench_loc, dy1] ]

#plateA   -SP
wbdict['features'][3]['coordinates'] = \
    [ [trench_loc, dy1], [trench_loc, dy2], [end_dist, dy2], [end_dist, dy1] ]

# Slab: trench location
# Slab location and orientation are defined by trench location and dip direction ("SlabTemp")
wbdict['features'][5]['coordinates'] = [[trench_loc, dy1], [trench_loc, dy2]]
wbdict['features'][5]['dip point']  = dip_point

# Slab location and orientation are defined by trench location and dip direction ("SlabComp")
wbdict['features'][6]['coordinates'] = [[trench_loc, dy1], [trench_loc, dy2]]
wbdict['features'][6]['dip point']  = dip_point

In [ ]:
# Plate and slab velocities for half-space and mass conserving temperature

#Get subducting plate's age at trench from get_trench_perp_profiles_regional.ipynb
#age_of_SP=103.7 #for K24 profile

age_of_SP = float(input('Age of Plate A at trench in million years: '))


In [ ]:
#Get observed speed of subducting plate A
#spr_vel_slab = 0.1107 #m/yr MORVEL
vel_slab = float(input('Velocity of Plate A in cm/yr: '))

In [ ]:
#OP ridge spreading rate 
#For this profile:(from literature Parson and Wright (1995): 0.045 (P30))
#spr_velBC = 0.045
spr_velBC = float(input('Spreading rate at B/C ridge m/yr: '))

In [ ]:
#This cell calculates the spreading velocity needed for Plate A 
#based on the age of the plate at the trench input above, and the the trench distance away

deg2km=111.11 #km/deg
km2m=1e3 #m/km
yr2sec=3.15e7 #s/y

age_at_trenchA=age_of_SP*1e6 #years
spr_vel_slab = vel_slab*1e-2 #m

#calculate coordinates of ridge location for Plate A and subducting slab based on observed ages and velocities
#distance from trench to ridge
distslab=spr_vel_slab*age_at_trenchA #meters
#convert km to degrees
distslabdeg=distslab/km2m/deg2km  
#putting the distance into a coordinate for my model domain reference frame
# these two are added together: trench is at trench_loc and the ridge is added with the length of the plate
slabridge=trench_loc+distslabdeg
print(slabridge)
#check with age calc
slabagecheck=distslab/spr_vel_slab
print(slabagecheck)

In [ ]:
wbdict['features'][0]['temperature models'][0]['spreading velocity'] = spr_velBC
wbdict['features'][1]['temperature models'][0]['spreading velocity'] = spr_velBC
wbdict['features'][2]['temperature models'][0]['spreading velocity'] = spr_velBC
wbdict['features'][3]['temperature models'][0]['spreading velocity'] = spr_vel_slab
wbdict['features'][4]['temperature models'][0]['spreading velocity'] = spr_vel_slab

# Locations of ridges needed for half space cooling models of both plates and slab
# These are start and end points on map
ridge_coordsBC = [[[ridgeBC_loc, dy1], [ridgeBC_loc, dy2]]]
ridge_coordsA = [[[trench_loc, dy3], [end_dist, dy3]]]
ridge_for_slab = [[[slabridge, dy1], [slabridge, dy2]]]

wbdict['features'][0]['temperature models'][0]['ridge coordinates'] = ridge_coordsBC
wbdict['features'][1]['temperature models'][0]['ridge coordinates'] = ridge_coordsBC
wbdict['features'][2]['temperature models'][0]['ridge coordinates'] = ridge_coordsBC
wbdict['features'][3]['temperature models'][0]['ridge coordinates'] = ridge_for_slab
wbdict['features'][4]['temperature models'][0]['ridge coordinates'] = ridge_for_slab
#wbdict['features'][2]['temperature models'][0]['ridge coordinates'] = ridge_coordsA

#########%%%%%%%%%REST OF THIS CELL NEEDS TO BE DOUBLE CHECKED for subduction direction%%%%%%
# Ridge and spreading velocity for slab depends on dip direction
#if dip_direction == 0:
#    ridge_for_slab = ridge_coordsA # dips north or west
#    spr_vel_slab = spr_velA
#else:
#    ridge_for_slab = ridge_coordsBC # dips south or east
#    spr_vel_slab = spr_velBC


wbdict['features'][5]['temperature models'][0]['plate velocity'] = spr_vel_slab
wbdict['features'][5]['temperature models'][0]['ridge coordinates'] = ridge_for_slab

In [ ]:
# General Material Parameters
surface_temperature = 273 # K
bottom_temperature = -1 # use adiabatic temperature gradient

wbdict['surface temperature'] = surface_temperature
wbdict['potential mantle temperature'] = 1573 # K
wbdict['thermal expansion coefficient'] = 3.1e-5 # 1/K
wbdict['specific heat'] = 1250. # J/(kg K)
wbdict['thermal diffusivity'] = 1.0e-6 # m^2/s
#wbdict['force surface temperature'] = 'false'
# Slab Mass Conserving (also uses general parameters above, so must be consistent)
wbdict['features'][5]['temperature models'][0]['density'] = 3300.0 # kg/m^3
wbdict['features'][5]['temperature models'][0]['thermal conductivity'] = 3.3 # W/m/K
#wbdict['features'][3]['temperature models'][0]['adiabatic heating'] = 'true' 

# Half-space Cooling Model and Mass COnserving: background temperature --> adiabatic or not?

wbdict['features'][0]['temperature models'][0]['top temperature'] = surface_temperature
wbdict['features'][0]['temperature models'][0]['bottom temperature'] = bottom_temperature
wbdict['features'][1]['temperature models'][0]['top temperature'] = surface_temperature
wbdict['features'][1]['temperature models'][0]['bottom temperature'] = bottom_temperature
wbdict['features'][3]['temperature models'][0]['top temperature'] = surface_temperature
wbdict['features'][3]['temperature models'][0]['bottom temperature'] = bottom_temperature

# Other specific Mass-conserving temperature parameters
wbdict['features'][5]['temperature models'][0]['coupling depth'] = 50e3 # km
wbdict['features'][5]['temperature models'][0]['forearc cooling factor'] = 15 
wbdict['features'][5]['temperature models'][0]['taper distance'] = 100e3 # km
theta = math.atan(5e4/1.59e5)
print(theta)

In [ ]:
# Depths for calculating temperature and composition
# Assumes plates and slab have both a crust and harzburgite layer
#Assumes plates B and C (OP2 and OP1) have same geometry

min_depth  = -10e3 # km Same for A, B, C
max_depthBC = 700e3 # km
max_depthA = 700e3 # km
max_crust = 7.5e3 # km Same for A, B, C
max_harz = 35.0e3 # km Same for A, B, C

# Plate C -OP1
wbdict['features'][0]['min depth'] = min_depth
wbdict['features'][0]['max depth'] = max_depthBC
wbdict['features'][0]['temperature models'][0]['min depth'] = min_depth
wbdict['features'][0]['temperature models'][0]['max depth'] = max_depthBC
wbdict['features'][0]['composition models'][0]['min depth'] = min_depth
wbdict['features'][0]['composition models'][0]['max depth'] = max_crust
wbdict['features'][0]['composition models'][1]['min depth'] = max_crust
wbdict['features'][0]['composition models'][1]['max depth'] = max_harz

# Plate B -OP2
wbdict['features'][1]['min depth'] = min_depth
wbdict['features'][1]['max depth'] = max_depthBC
wbdict['features'][1]['temperature models'][0]['min depth'] = min_depth
wbdict['features'][1]['temperature models'][0]['max depth'] = max_depthBC
wbdict['features'][1]['composition models'][0]['min depth'] = min_depth
wbdict['features'][1]['composition models'][0]['max depth'] = max_crust
wbdict['features'][1]['composition models'][1]['min depth'] = max_crust
wbdict['features'][1]['composition models'][1]['max depth'] = max_harz

#WC -OP
wbdict['features'][2]['min depth'] = min_depth
wbdict['features'][2]['max depth'] = max_depthBC
wbdict['features'][2]['temperature models'][0]['min depth'] = min_depth
wbdict['features'][2]['temperature models'][0]['max depth'] = max_depthBC
wbdict['features'][2]['composition models'][0]['min depth'] = min_depth
wbdict['features'][2]['composition models'][0]['max depth'] = max_crust
wbdict['features'][2]['composition models'][1]['min depth'] = max_crust
wbdict['features'][2]['composition models'][1]['max depth'] = max_harz

# Plate A -SP
wbdict['features'][3]['min depth'] = min_depth
wbdict['features'][3]['max depth'] = max_depthA
wbdict['features'][3]['temperature models'][0]['min depth'] = min_depth
wbdict['features'][3]['temperature models'][0]['max depth'] = max_depthA
wbdict['features'][3]['composition models'][0]['min depth'] = min_depth
wbdict['features'][3]['composition models'][0]['max depth'] = max_crust
wbdict['features'][3]['composition models'][1]['min depth'] = max_crust
wbdict['features'][3]['composition models'][1]['max depth'] = max_harz

#WC-SP
wbdict['features'][4]['min depth'] = min_depth
wbdict['features'][4]['max depth'] = max_depthA
wbdict['features'][4]['temperature models'][0]['min depth'] = min_depth
wbdict['features'][4]['temperature models'][0]['max depth'] = max_depthA
wbdict['features'][4]['composition models'][0]['min depth'] = min_depth
wbdict['features'][4]['composition models'][0]['max depth'] = max_crust
wbdict['features'][4]['composition models'][1]['min depth'] = max_crust
wbdict['features'][4]['composition models'][1]['max depth'] = max_harz

# Slab Mass Conserving Temperature "SlabTemp"
wbdict['features'][5]['temperature models'][0]['min distance slab top'] = -900e3 # km
wbdict['features'][5]['temperature models'][0]['max distance slab top'] = 700e3 # km

# Slab Mass Conserving Composition "SlabComp"
wbdict['features'][6]['composition models'][0]['max distance slab top'] = max_crust # km
wbdict['features'][6]['composition models'][1]['min distance slab top'] = max_crust # km
wbdict['features'][6]['composition models'][1]['max distance slab top'] = max_harz  #km

#####I don't think this bottom if-then statement is needed? thicknesses of crust is same regardless?
# Crustal thickness for slab depends on dip direction
#if dip_direction == 0: # dips north or west
#    wbdict['features'][2]['composition models'][0]['max distance slab top'] = max_crustB # km
    #wbdict['features'][2]['composition models'][1]['min distance slab top'] = max_crustB # km
    #wbdict['features'][2]['composition models'][1]['max distance slab top'] = max_harzB # km
#else: # dips south or east
#    wbdict['features'][2]['composition models'][0]['max distance slab top'] = max_crustA # km 
#    #wbdict['features'][2]['composition models'][1]['min distance slab top'] = max_crustA # km
#    #wbdict['features'][2]['composition models'][1]['max distance slab top'] = max_harzA # km


In [ ]:
# Here we want to read in the segment list created for the trench perpendicular profile
# across the slab 2.0 data.

file_prefix = "ker_1.0d_p017_az-71.236"

#read in segments for SlabTemp
# segfile_objectTemp = open('ker_0.7d_p030_az-65.505_wb_slab_segmentsTemp.json', 'r')
segfile_objectTemp = open(file_prefix + "_wb_slab_segmentsTempNew.json", 'r')
segdictTemp = json.load(segfile_objectTemp)
print('The slab segment file has been read in as a ', type(segdictTemp))

#read in segments for SlabComp
# segfile_objectComp = open('ker_0.7d_p030_az-65.505_wb_slab_segmentsComp.json','r')
segfile_objectComp = open(file_prefix + "_wb_slab_segmentsCompNew.json", 'r')
segdictComp = json.load(segfile_objectComp)
print('The slab segment file has been read in as a ', type(segdictComp))

wbdict['features'][5]['segments'] = segdictTemp['segments']
wbdict['features'][6]['segments'] = segdictComp['segments']

In [ ]:
# How to output the modified dictionary as a json file
# Serializing json
# QS Becky: position of the sections

wbjson_object = json.dumps(wbdict, indent = 2)

ofile = file_prefix + "_CplatesextendREVISED.json"
  
# Writing to sample.json
with open(ofile, "w") as outfile:
    outfile.write(wbjson_object)

print("Write file %s" % ofile)